In [1]:
#載入github上的json檔
import pandas as pd
df = pd.read_json('raw.json')
keep_dataframe = df[['jobData','applyData','job_id','cust_id']]

In [2]:
#傳回相同元素的index
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

In [6]:
#應徵分析x變數
complete_data = []
complete_data_percent = []

an_index = 0

for unit_data in range(len(keep_dataframe['jobData'])):
    row_data = []
    row_data_percent = []
    
    if unit_data in dead_row:
        continue
    
    #index
    row_data.append(an_index)
    
    ###應徵分析###
    row_data.append(float(keep_dataframe['applyData'][unit_data]['sex']['0']['percent'])*0.01) #男生比例
    
    #科系
    major = []
    major_percent = {}
    del keep_dataframe['applyData'][unit_data]['major']['update_time']
    del keep_dataframe['applyData'][unit_data]['major']['total']
    for i in range(len(keep_dataframe['applyData'][unit_data]['major'])):
        major.append(keep_dataframe['applyData'][unit_data]['major'][str(i)]['majorName'])
        major_percent[keep_dataframe['applyData'][unit_data]['major'][str(i)]['majorName']] = float(keep_dataframe['applyData'][unit_data]['major'][str(i)]['percent']) * 0.01
    row_data.append(major)
    row_data_percent.append(major_percent) 
    
    #學歷
    education = []
    education_percent = {}
    del keep_dataframe['applyData'][unit_data]['edu']['update_time']
    del keep_dataframe['applyData'][unit_data]['edu']['total']
    for i in range(len(keep_dataframe['applyData'][unit_data]['edu'])):
        education.append(keep_dataframe['applyData'][unit_data]['edu'][str(i)]['eduName'])
        education_percent[keep_dataframe['applyData'][unit_data]['edu'][str(i)]['eduName']] = float(keep_dataframe['applyData'][unit_data]['edu'][str(i)]['percent'])* 0.01
    row_data.append(education)
    row_data_percent.append(education_percent)    
    
    #年齡
    yearage = []
    yearage_percent = {}
    del keep_dataframe['applyData'][unit_data]['yearRange']['update_time']
    del keep_dataframe['applyData'][unit_data]['yearRange']['total']
    for i in range(len(keep_dataframe['applyData'][unit_data]['yearRange'])):
        yearage.append(keep_dataframe['applyData'][unit_data]['yearRange'][str(i)]['yearRangeName'])
        yearage_percent[keep_dataframe['applyData'][unit_data]['yearRange'][str(i)]['yearRangeName']] = float(keep_dataframe['applyData'][unit_data]['yearRange'][str(i)]['percent'])* 0.01
    row_data.append(yearage)
    row_data_percent.append(yearage_percent)
    
    #工作經驗
    exp = []
    exp_percent = {}
    del keep_dataframe['applyData'][unit_data]['exp']['update_time']
    del keep_dataframe['applyData'][unit_data]['exp']['total']
    for i in range(len(keep_dataframe['applyData'][unit_data]['exp'])):
        exp.append(keep_dataframe['applyData'][unit_data]['exp'][str(i)]['expName'])
        exp_percent[keep_dataframe['applyData'][unit_data]['exp'][str(i)]['expName']] = float(keep_dataframe['applyData'][unit_data]['exp'][str(i)]['percent'])* 0.01
    row_data.append(exp)
    row_data_percent.append(exp_percent)
    
    #語言
    selected_lan = ['英文','日文','韓文','德文','西班牙文']
    del keep_dataframe['applyData'][unit_data]['language']['update_time']
    del keep_dataframe['applyData'][unit_data]['language']['total']

    lan_list={}
    for i in range(len(keep_dataframe['applyData'][unit_data]['language'])):
        if keep_dataframe['applyData'][unit_data]['language'][str(i)]['langName'] in selected_lan:
            if type(keep_dataframe['applyData'][unit_data]['language'][str(i)]['level']) is list:
                keep_dataframe['applyData'][unit_data]['language'][str(i)]['level'] = {'0':keep_dataframe['applyData'][unit_data]['language'][str(i)]['level'][0]}
            if keep_dataframe['applyData'][unit_data]['language'][str(i)]['langName'] in lan_list:
                lan_list[keep_dataframe['applyData'][unit_data]['language'][str(i)]['langName']].append(keep_dataframe['applyData'][unit_data]['language'][str(i)]['level'])
            else:
                lan_list[keep_dataframe['applyData'][unit_data]['language'][str(i)]['langName']] = [keep_dataframe['applyData'][unit_data]['language'][str(i)]['level']]
        else:
            if type(keep_dataframe['applyData'][unit_data]['language'][str(i)]['level']) is list:
                keep_dataframe['applyData'][unit_data]['language'][str(i)]['level'] = {'0':keep_dataframe['applyData'][unit_data]['language'][str(i)]['level'][0]}
            if '其他' in lan_list:
                lan_list['其他'].append(keep_dataframe['applyData'][unit_data]['language'][str(i)]['level'])
            else:
                lan_list['其他'] = [keep_dataframe['applyData'][unit_data]['language'][str(i)]['level']]
    all_language = {}
    key = list(lan_list.keys())
    levels = ['精通','中等','略懂','不會']
    for k in key:
        inner_level = []
        inner_percent = []
        for inner in range(len(lan_list[k])):
            #為一個dictionary
            inner_key = list(lan_list[k][inner].keys())
            for y in inner_key:
                inner_level.append(lan_list[k][inner][y]['levelName'])
                inner_percent.append(float(lan_list[k][inner][y]['percent']))
        new_inner_percent = []
        new_inner_level = []
        for lev in levels: #算語言_level的percent
            dup_index = list_duplicates_of(inner_level, lev)
            if dup_index != []:
                new_inner_level.append(lev)
                new_per = 0
                for c in dup_index:
                    new_per += inner_percent[c]
                new_inner_percent.append(new_per)
        count = 0
        for new_level in new_inner_level:
            all_language[k + '_' + new_level] = new_inner_percent[count] * 0.01
            count += 1

    row_data_percent.append(all_language)
    row_data.append(list(all_language.keys()))
    
    #工作技能
    skill = []
    skill_percent = {}
    del keep_dataframe['applyData'][unit_data]['skill']['update_time']
    del keep_dataframe['applyData'][unit_data]['skill']['total']
    for i in range(len(keep_dataframe['applyData'][unit_data]['skill'])):
        if float(keep_dataframe['applyData'][unit_data]['skill'][str(i)]['percent']) >= 50:
            skill.append(keep_dataframe['applyData'][unit_data]['skill'][str(i)]['skillName'])
            skill_percent[keep_dataframe['applyData'][unit_data]['skill'][str(i)]['skillName']] = float(keep_dataframe['applyData'][unit_data]['skill'][str(i)]['percent']) * 0.01
    row_data.append(skill)
    row_data_percent.append(skill_percent)
    
    #證照
    cert = []
    cert_percent = {}
    del keep_dataframe['applyData'][unit_data]['cert']['update_time']
    del keep_dataframe['applyData'][unit_data]['cert']['total']
    for i in range(len(keep_dataframe['applyData'][unit_data]['cert'])):
        if float(keep_dataframe['applyData'][unit_data]['cert'][str(i)]['percent']) >= 50:
            cert.append(keep_dataframe['applyData'][unit_data]['cert'][str(i)]['certName'])
            cert_percent[keep_dataframe['applyData'][unit_data]['cert'][str(i)]['certName']] = float(keep_dataframe['applyData'][unit_data]['cert'][str(i)]['percent'])* 0.01
    row_data.append(cert)
    row_data_percent.append(cert_percent)
            
    complete_data_percent.append(row_data_percent)
    complete_data.append(row_data)
    an_index += 1

In [7]:
column_name_apply = ['id','apply_male_ratio','apply_major','apply_education','apply_age','apply_exp','apply_lang','apply_skill','apply_cert']
column_name_percent_apply = ['major_percent','education_percent','yearage_percent','exp_percent','lang_percent','skill_percent','cert_percent']

In [ ]:
none_value = []
for t in range(len(complete_data_percent)):
    if {} in complete_data_percent[t]:
        none_value.append(t)

In [ ]:
len(none_value)

In [8]:
df_apply = pd.DataFrame(complete_data,columns=column_name_apply)
df_percent_apply = pd.DataFrame(complete_data_percent,columns=column_name_percent_apply)

In [ ]:
df_percent_apply.to_csv('apply_percent.csv')

In [9]:
df_apply_long = df_apply.explode('apply_major')
one_hot_apply = pd.get_dummies(df_apply_long,columns=['apply_major'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [10]:
one_hot_apply = one_hot_apply.explode('apply_education')
one_hot_apply = pd.get_dummies(one_hot_apply,columns=['apply_education'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [11]:
one_hot_apply = one_hot_apply.explode('apply_age')
one_hot_apply = pd.get_dummies(one_hot_apply,columns=['apply_age'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [12]:
one_hot_apply = one_hot_apply.explode('apply_exp')
one_hot_apply = pd.get_dummies(one_hot_apply,columns=['apply_exp'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [13]:
one_hot_apply = one_hot_apply.explode('apply_lang')
one_hot_apply = pd.get_dummies(one_hot_apply,columns=['apply_lang'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [14]:
one_hot_apply = one_hot_apply.explode('apply_skill')
one_hot_apply = pd.get_dummies(one_hot_apply,columns=['apply_skill'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [15]:
one_hot_apply = one_hot_apply.explode('apply_cert')
one_hot_apply = pd.get_dummies(one_hot_apply,columns=['apply_cert'])
one_hot_apply = one_hot_apply.groupby('id').max().reset_index()

In [16]:
#刪掉薪資缺值的row
one_hot_apply = one_hot_apply.drop(labels=record_delete_row , axis=0)
df_percent_apply = df_percent_apply.drop(labels=record_delete_row , axis=0)

In [ ]:
list_percent = df_percent_apply.values.tolist()

In [18]:
column_name_ = ['apply_major','apply_education','apply_age','apply_exp','apply_lang','apply_skill','apply_cert']

In [19]:
column_name_percent_apply = ['major_percent','education_percent','yearage_percent','exp_percent','lang_percent','skill_percent','cert_percent']

In [ ]:
for data_row_column in range(10000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[:10000].to_csv('10000_apply.csv')

In [ ]:
for data_row_column in range(10000, 20000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[10000:20000].to_csv('20000_apply.csv')

In [ ]:
for data_row_column in range(20000,30000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[20000:30000].to_csv('30000_apply.csv')

In [ ]:
for data_row_column in range(30000,40000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[30000:40000].to_csv('40000_apply.csv')

In [ ]:
for data_row_column in range(40000,50000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[40000:50000].to_csv('50000_apply.csv')

In [ ]:
for data_row_column in range(50000,60000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[50000:60000].to_csv('60000_apply.csv')

In [ ]:
for data_row_column in range(60000,70000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[60000:70000].to_csv('70000_apply.csv')

In [ ]:
for data_row_column in range(70000,80000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[70000:80000].to_csv('80000_apply.csv')

In [ ]:
for data_row_column in range(80000,90000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[80000:90000].to_csv('90000_apply.csv')

In [ ]:
for data_row_column in range(90000,100000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [ ]:
one_hot_apply[90000:100000].to_csv('100000_apply.csv')

In [20]:
for data_row_column in range(100000,110000):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [21]:
one_hot_apply[100000:110000].to_csv('110000_apply.csv')

In [22]:
for data_row_column in range(110000,len(one_hot_apply)):
    count = 0
    for name in column_name_percent_apply:
        try:
            name_int = df_percent_apply.loc[data_row_column][name]
            for point in name_int.keys():
                one_hot_apply.loc[data_row_column:data_row_column+1, [column_name_[count] + '_' + point]] = [name_int[point]]
        except:
            pass
        count += 1

In [23]:
one_hot_apply[110000:len(one_hot_apply)].to_csv('last_apply.csv')

In [ ]:
one_hot_apply = one_hot_apply.drop(['id'], axis=1)

In [3]:
#薪水
y_all = []
dead_row = []
for l in range(len(keep_dataframe['jobData'])):
    if len(keep_dataframe['jobData'][l]) < 17:
        dead_row.append(l)
        continue
    one_y = []
    salaryMin = keep_dataframe['jobData'][l]['jobDetail']['salaryMin']
    if salaryMin < 10000:
        salaryMin = None
    salaryMax = keep_dataframe['jobData'][l]['jobDetail']['salaryMax']
    if salaryMax == 9999999:
        salaryMax = None
    one_y = [salaryMin, salaryMax]
    y_all.append(one_y)

y_column_name = ['salaryMin','salaryMax']
y_train = pd.DataFrame(y_all, columns=y_column_name)

In [4]:
record_delete_row = []
for o in range(len(keep_dataframe['jobData'])-len(dead_row)):
    if 0 in y_all[o]:
        record_delete_row.append(o)
        continue
    if None in y_all[o]:
        record_delete_row.append(o)
        continue

In [5]:
y_train = y_train.drop(labels=record_delete_row , axis=0)
y_train

,salaryMin,salaryMax
0,36900.0,40000.0
1,45000.0,70000.0
3,30000.0,80000.0
4,35000.0,40000.0
6,50000.0,70000.0
...,...,...
192906,32000.0,35000.0
192907,33400.0,45400.0
192908,45000.0,65000.0
192909,35000.0,45000.0


In [26]:
print(y_train['salaryMax'].mean())
print(y_train['salaryMin'].mean())

55701.006330113145
35035.00408224636


In [27]:
print(y_train['salaryMax'].std())
print(y_train['salaryMin'].std())

163668.97477900377
55329.46312593438


In [ ]:
y_train.to_csv('before_standard_y.csv')